In [1]:
print("hello")

hello


In [ ]:
%pwd

'd:\\projects\\medical-chatbot\\research'

In [ ]:

import os
os.chdir("../")

In [2]:
%pwd

'/content'

In [4]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [5]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# load data

In [13]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 2.9 MB/s eta 0:00:00


In [14]:
def extract_pdf_files(data_path):
    # here in data folder you can add more pdfs too, loader will load all of them
    loader=DirectoryLoader(
        data_path,glob="*.pdf",loader_cls=PyPDFLoader
    )

    documents=loader.load()
    return documents


In [15]:
extracted_data=extract_pdf_files("sample_data")
# extracted_data
# langchain returning the document format

In [16]:
len(extracted_data) #number of pages

637

In [17]:
# page content is needed and it is present in document that we get
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs:List[Document])->List[Document]:
    """
    Given a list of documents, i wnat to return a new list of documents that only contains the source and the page_content
    """

    minimal_docs:List[Document]=[]
    for doc in docs:
        src=doc.metadata.get('source')
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source":src}
            )
        )

    return minimal_docs



In [18]:
extracted_text=filter_to_minimal_docs(extracted_data)

# chunkning

In [19]:
def text_splitter(extracted_text):
    # here create the chunks of the extracted data
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
        # length_function=len
    )

    chunked_text=text_splitter.split_documents(extracted_text)
    return chunked_text

In [23]:
len(text_splitter(extracted_text))
texts_chunk=text_splitter(extracted_text)

In [24]:
len(texts_chunk)

5859

# do embeddings of the chunks

In [26]:
# hugging face embedding model
from langchain.embeddings import HuggingFaceEmbeddings

# download the pretrained embedding model we want to use

def download_embedding_model():
    """
    Download the model and return the hugging face embedddings
    """
    model_name="sentence-transformers/all-MiniLM-L6-v2"
    embeddings=HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

In [27]:
embeddings=download_embedding_model()

In [28]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

texting the embedding of the model

In [36]:
print(len(list(embeddings.embed_query("hello i am dhruv"))))

384


In [51]:
import os

In [ ]:
# store the embeddings in the vector database
# access the pinecone and open ai api key  
from dotenv import load_dotenv
import os

load_dotenv()
PINECONE_API_KEY=os.getenv('PINECONE_API_KEY')
OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')

os.environ['PINECONE_API_KEY']=PINECONE_API_KEY
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [31]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [43]:
# import pinecone
from pinecone import Pinecone
pinecone_api_key=PINECONE_API_KEY

# authenticate pinecone accrount and create a client
pinecone_client=Pinecone(api_key=pinecone_api_key)


In [44]:
pinecone_client

In [45]:
# create index or database
from pinecone import  ServerlessSpec
index_name="medical-chatbot"

if not pinecone_client.has_index(index_name):
  pinecone_client.create_index(
      name=index_name,
      dimension=384,# dimenstion of the embeddings that we got above
      metric='cosine',
      spec=ServerlessSpec(cloud='aws',region='us-east-1')
  )


# creating the index, you can check it creates a new index in db
index=pinecone_client.Index(index_name)

In [39]:
!pip install langchain_pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.1 MB/s eta 0:00:00


In [40]:
!pip install langchain_openai

In [54]:
# now push the embeddings

from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

# creating embeddings using hf sentence transformer
docSearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    index=index,
    embedding=embeddings
)


In [55]:
# load a exisiting index and continue the work
# from langchain_pinecone import PineconeVectorStore

# existing_docSearch=PineconeVectorStore.from_existing_index(
#     index_name=index_name,
#     embedding=embeddings,
# )


# add new documents or new data to the existing knowledge base

In [56]:
# if we want to add more data t the existing pinecone index to increase the kb
# creating a dummy documnet
dummy=Document(
    page_content='Lionel Messi is the best soccer player',
    metadata={'source':'manual_entry'}
)

In [57]:
docSearch.add_documents([dummy])

['6a717b68-6ced-4a2d-831b-2e2e3fdbe247']

# now create a retriever llm model

In [58]:
retriever=docSearch.as_retriever(
    search_type='similarity',
    search_kwargs={"k":4}, # overlapping chunks number, get top 4 retrived sentences
)

In [60]:
question='who is the best soccer player'
retrieved_docs=retriever.invoke(question)
# retrieved_docs
for doc in retrieved_docs:
  print(doc.page_content[:40])

Lionel Messi is the best soccer player
be understood are among our most basic n
(Zovirax), famciclovir (Famvir), and val
United States. Milieu therapies (or envi


In [62]:
# i want to refine the resposne with the llm
from langchain_openai import ChatOpenAI

chatmodel=ChatOpenAI(
    model='gpt-4o'
)

In [67]:
# create a chain, to setup the rag pipleine
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt_template=(
    "You are a Medical chatbot assistant that is trained for question-answering tasks. "
    "Use the following pieces of retrived context to answer the user's question properly."
    " If you do not know the answer, apologize for the inconvenience, do not give"
    " misinformation. say that you do not know the answer. "
    "Use the retrieved context properly and keep the answers concise \n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages( # Use from_messages for multiple roles

        [ # Pass a list of tuples
            ("system",prompt_template), #the system prompt
            ("human","{input}") #input from the user
        ]

)

In [ ]:
# create a chain
# ChatPromptTemplate = defines how to ask

# create_stuff_documents_chain = defines how to insert docs into the prompt

# create_retrieval_chain = glues retriever + doc chain into a single RAG pipeline

question_answer_chain=create_stuff_documents_chain(chatmodel,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [78]:
que="who is dhruv"

In [79]:
print(rag_chain.invoke({"input":que})['answer'])

I'm sorry, but I do not have any information on someone named Dhruv based on the provided context. If you have more specific details or context regarding who Dhruv might be, feel free to share.
